In [52]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, Dropout, Activation, Reshape, Lambda, Merge, Embedding
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.utils import np_utils
from keras import backend as K
import theano

In [9]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [3]:
batch_size = 128
nb_classes = 10
nb_epoch = 2
# number of convolutional filters to use
nb_filters = 16
# size of pooling area for max pooling
pool_size = (2, 1)
# convolution kernel size
kernel_size = (5, 5)

In [4]:
import theano.tensor as T
from theano import function
from theano import pp
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = function([x, y], z)
print(f(2, 3))
print(pp(z))
z.eval({x : 2, y : 3})

5.0
(x + y)


array(5.0)

In [11]:
###important indexing method!!
b=np.random.rand(3, 3)
i = [(1,0),(0,1),(2,1)]
i=np.asarray(i)
b[i[:,0],i[:,1]]

np.random.seed(12)
a = np.random.choice(1000, 120, replace=False)
a = a.reshape((2,3,4,5))
print(a.shape)
print(a)

x = T.dtensor4('x')

shape = x.shape[2]
getshape=function([x],shape)
print(getshape(a))

args1 = T.max(x, axis=2)
maxvalue = function([x],args1)
print(maxvalue(a))

args = T.argsort(x, axis=2)
print(function([x],args)(a))

k=2
args = T.argsort(x, axis=2)
kargs = args[:,:,-k:,:]
kargs = T.sort(kargs, axis=2)
# z = kargs.shape
maxf = function([x],kargs)
print(maxf(a))


mymax = T.iscalar('mymax')
rep = T.iscalar('rep')
myrange = T.arange(mymax).repeat(rep)
getindex = function([mymax,rep], myrange)
print(getindex(5,2))
##output [0 0 1 1 2 2 3 3 4 4]

mymax = T.iscalar('mymax')
rep = T.iscalar('rep')
res1 = T.iscalar('res1')
res2 = T.iscalar('res2')
myrange = T.arange(mymax).repeat(rep).reshape((res1,res2))
getindex2 = function([mymax,rep,res1,res2], myrange)
print(getindex2(5,2,2,5))
##output [[0 0 1 1 2]
# [2 3 3 4 4]]
print(getindex2(5,2,1,-1)) 
## output [[0 0 1 1 2 2 3 3 4 4]]

(2, 3, 4, 5)
[[[[518 871 797 274 325]
   [827 766 832 931 368]
   [562 381 507 506   3]
   [842 136 938 854 190]]

  [[923 394 122 315 320]
   [109 307 488 424 744]
   [321 997 668 311 620]
   [792  79 228 657 101]]

  [[598 918 168 495 734]
   [596 207  78  50 966]
   [161 822 937 572 244]
   [102 607 387 324 397]]]


 [[[592 504 438 655 858]
   [476 526 421 514 365]
   [652 172 664 847 673]
   [ 71 829  23 517 393]]

  [[771 743 408 879 351]
   [224 739 193 547 732]
   [995 907 258 519 235]
   [403 886 542  29 116]]

  [[919 759 704 431 694]
   [868 450 959 823 563]
   [804 612 509 803 541]
   [270 725 129 216 301]]]]
4
[[[ 842.  871.  938.  931.  368.]
  [ 923.  997.  668.  657.  744.]
  [ 598.  918.  937.  572.  966.]]

 [[ 652.  829.  664.  847.  858.]
  [ 995.  907.  542.  879.  732.]
  [ 919.  759.  959.  823.  694.]]]
[[[[0 3 2 0 2]
   [2 2 0 2 3]
   [1 1 1 3 0]
   [3 0 3 1 1]]

  [[1 3 0 2 3]
   [2 1 3 0 0]
   [3 0 1 1 2]
   [0 2 2 3 1]]

  [[3 1 1 1 2]
   [2 3 0 3 3]
   [1 2 

In [37]:
np.random.seed(12)
a = np.random.choice(1000, 120, replace=False)
a = a.reshape((2,3,4,5))
print(a.shape)
print(a)

####axis = 3
# asort = np.argsort(a,axis=3)
# ksort = np.sort(asort[:,:,:,-k:],axis=3)

# dim0 = np.asarray(range(2)).repeat(3*4*2)
# dim1 = np.asarray(range(3)).repeat(4*2).reshape((1,-1)).repeat(2,axis=0).flatten()
# dim2 = np.asarray(range(4)).repeat(2).reshape((1,-1)).repeat(2*3,axis=0).flatten()
# dim3 = ksort.flatten()

#### axis = 2
asort = np.argsort(a,axis=2)
ksort = np.sort(asort[:,:,-k:,:],axis=2)

dim0 = np.asarray(range(2)).repeat(3*2*5)
dim1 = np.asarray(range(3)).repeat(2*5).reshape((1,-1)).repeat(2,axis=0).flatten()
dim2 = ksort.flatten()
dim3 = np.asarray(range(5)).reshape((1,-1)).repeat(2*3*2,axis=0).flatten()

print(a[dim0,dim1,dim2,dim3].reshape((2,3,2,5)))

(2, 3, 4, 5)
[[[[518 871 797 274 325]
   [827 766 832 931 368]
   [562 381 507 506   3]
   [842 136 938 854 190]]

  [[923 394 122 315 320]
   [109 307 488 424 744]
   [321 997 668 311 620]
   [792  79 228 657 101]]

  [[598 918 168 495 734]
   [596 207  78  50 966]
   [161 822 937 572 244]
   [102 607 387 324 397]]]


 [[[592 504 438 655 858]
   [476 526 421 514 365]
   [652 172 664 847 673]
   [ 71 829  23 517 393]]

  [[771 743 408 879 351]
   [224 739 193 547 732]
   [995 907 258 519 235]
   [403 886 542  29 116]]

  [[919 759 704 431 694]
   [868 450 959 823 563]
   [804 612 509 803 541]
   [270 725 129 216 301]]]]
[[[[827 871 832 931 325]
   [842 766 938 854 368]]

  [[923 394 488 424 744]
   [792 997 668 657 620]]

  [[598 918 937 495 734]
   [596 822 387 572 966]]]


 [[[592 526 438 655 858]
   [652 829 664 847 673]]

  [[771 907 408 879 351]
   [995 886 542 547 732]]

  [[919 759 704 823 694]
   [868 725 959 803 563]]]]


In [49]:
x = T.dtensor4('x')
k=2
sorted_values = T.argsort(x,axis=2)
topmax_indexes = sorted_values[:,:,-k:,:]
topmax_indexes_sorted = T.sort(topmax_indexes,axis=2)    
dim0 = T.arange(0,x.shape[0]).repeat(x.shape[1]*k*x.shape[3])
dim1 = T.arange(0,x.shape[1]).repeat(k*x.shape[3]).reshape((1,-1)).repeat(x.shape[0],axis=0).flatten()
dim2 = topmax_indexes_sorted.flatten()
dim3 = T.arange(0,x.shape[3]).reshape((1,-1)).repeat(x.shape[0]*x.shape[1]*k,axis=0).flatten()
res=x[dim0,dim1,dim2,dim3].reshape((x.shape[0], x.shape[1], k, x.shape[3]))

f = theano.function([x],[dim0,dim1,dim2,dim3])
print(f(a))
xx=x.flatten()
f1 = theano.function([x],xx)
f2 = theano.function([x],dim3+dim0*3*2*5+dim1*2*5+dim2*5)
f3 = theano.function([x],xx[dim3+dim0*3*2*5+dim1*2*5+dim2*5].reshape((2,3,2,5)))
print(f1(a))
print(f2(a))
print(f3(a))
print(a)
theano.printing.debugprint(f3(a))

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), array([1, 0, 1, 1, 0, 3, 1, 3, 3, 1, 0, 0, 1, 1, 1, 3, 2, 2, 3, 2, 0, 0, 2,
       0, 0, 1, 2, 3, 2, 1, 0, 1, 0, 0, 0, 2, 3, 2, 2, 2, 0, 2, 0, 0, 0, 2,
       3, 3, 1, 1, 0, 0, 0, 1, 0, 1, 3, 1, 2, 1]), array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2,
       3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0,
       1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4])]
[ 518.  871.  797.  274.  325.  827.  766.  832.  931.  368.  562.  381.
  507.  506.    3.  842.  136.  938.  854.  190.  923.  394.  122.  315.
  320.  109.  307.  488.  424.  744.  321

In [56]:
def themax(x):
    return T.max(x, axis=2)

###axis = 2
def k_max(x):
#     k=T.iscalar('k')
    k = x.shape[2]/2
    sorted_values = T.argsort(x,axis=2)
    topmax_indexes = sorted_values[:,:,-k:,:]
    # sort indexes so that we keep the correct order within the sentence
    topmax_indexes_sorted = T.sort(topmax_indexes,axis=2)    
    #given that topmax only gives the index of the third dimension, we need to generate the other 3 dimensions
    dim0 = T.arange(0,x.shape[0]).repeat(x.shape[1]*k*x.shape[3])
    dim1 = T.arange(0,x.shape[1]).repeat(k*x.shape[3]).reshape((1,-1)).repeat(x.shape[0],axis=0).flatten()
    dim2 = topmax_indexes_sorted.flatten()
    dim3 = T.arange(0,x.shape[3]).reshape((1,-1)).repeat(x.shape[0]*x.shape[1]*k,axis=0).flatten()
    return x[dim0,dim1,dim2,dim3].reshape((x.shape[0], x.shape[1], k, x.shape[3]))

##aixs = 3
def k_max2(x):
#     k=T.iscalar('k')
#     k=x.shape[3]/2
    k=5
    sorted_values = T.argsort(x,axis=3)
    topmax_indexes = sorted_values[:,:,:,-k:]
    # sort indexes so that we keep the correct order within the sentence
    topmax_indexes_sorted = T.sort(topmax_indexes)    
    #given that topmax only gives the index of the third dimension, we need to generate the other 3 dimensions
    dim0 = T.arange(0,x.shape[0]).repeat(x.shape[1]*x.shape[2]*k)
    dim1 = T.arange(0,x.shape[1]).repeat(k*x.shape[2]).reshape((1,-1)).repeat(x.shape[0],axis=0).flatten()
    dim2 = T.arange(0,x.shape[2]).repeat(k).reshape((1,-1)).repeat(x.shape[0]*x.shape[1],axis=0).flatten()
    dim3 = topmax_indexes_sorted.flatten()
    return x[dim0,dim1,dim2,dim3].reshape((x.shape[0], x.shape[1], x.shape[2],k))

def output_shape_f(shape):
    return (shape[0],shape[1],shape[2]/2,shape[3])    

def output_shape_f2(shape):
    return (shape[0],shape[1],shape[2],5)  

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
kmax_layer = Lambda(function=k_max, output_shape=output_shape_f)
model.add(kmax_layer)
# model.add(AveragePooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: DeprecationWarning: Division of two integer types with x / y is deprecated, please use x // y for an integer division.


Train on 60000 samples, validate on 10000 samples
Epoch 1/2
12928/60000 [=====>........................] - ETA: 19s - loss: 1.4669 - acc: 0.4982

KeyboardInterrupt: 

In [75]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 19s - loss: 0.5924 - acc: 0.8063 - val_loss: 0.1169 - val_acc: 0.9638
Epoch 2/2
60000/60000 [==============================] - 18s - loss: 0.2141 - acc: 0.9367 - val_loss: 0.0880 - val_acc: 0.9718
Test score: 0.0879899788989
Test accuracy: 0.9718


In [20]:
def get_activations(model, layer, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output])
    activations = get_activations([X_batch,0])
    return activations

In [21]:
X_train[0].shape
model.layers[3]

In [22]:

test=X_train[:3]
print(model.layers[0])
print(model.layers[5])
print(model.layers[0].input)
print(model.layers[0].input.type)
res=get_activations(model, 5, test)[0]
print(test.shape)
print(res.shape)
print(res[0][0][2])
res2=get_activations(model, 6, test)[0]
print(res2[0][0][2])

convolution2d_input_2
TensorType(float32, 4D)
(3, 1, 28, 28)
(3, 16, 8, 20)
[ 0.01824202  0.          0.          0.          0.          0.07622255
  0.26356173  0.22608617  0.17033559  0.26586303  0.44771817  0.59780973
  1.21221602  1.80400455  2.54239893  2.69711947  2.54329491  2.21155906
  1.71171498  1.17737687]
[ 0.3264716   0.46496376  0.57620698  0.65903562  0.99940348  1.42767763
  1.59193778  1.55196595  0.853104    0.63897258  0.30281779  0.09640662
  0.          0.          0.          1.4456352   1.42369044  1.17533433
  0.83574235  0.47179934]


In [52]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 16, 24, 24)    416         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
activation_11 (Activation)       (None, 16, 24, 24)    0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 16, 20, 20)    6416        activation_11[0][0]              
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 16, 20, 20)    0           convolution2d_8[0][0]            
___________________________________________________________________________________________